In [1]:
import trimet_plan_game_reset as tpg
import geopandas as gpd

In [6]:
origin_str = "45.495953, -122.557314"
# "45.495953, -122.557314"
# "45.582637, -122.754038"
# "45.512908, -122.711987"
destination_str = "45.517341, -122.401294"
# "45.517341, -122.401294"
# "45.512894, -122.656701"
#  "45.510896, -122.611478"
planner_response = tpg.call_planner(origin_str, destination_str)

In [7]:
itineraries_df = tpg.get_itinerary_paths(planner_response)
itineraries_df[itineraries_df['route_id']!='WALK'].groupby(['itin_idx','route_id']).agg(leg_count=('leg_id','count')).reset_index()

,itin_idx,route_id,leg_count
0,0,20,1
1,0,9,1
2,1,20,1
3,1,73,1
4,1,9,1
5,2,20,1
6,2,9,1
7,3,20,1
8,3,9,1
9,4,20,1


In [8]:
itineraries_reduced, itinerary_routes_reduced = tpg.create_intinerary_gdf_and_reduce(itineraries_df)

In [9]:
itineraries_reduced

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_id_combo
0,0,3478,389,2171,918,512.015194,0,WALK,WALK,,Origin,4562,SE Powell & 102nd,"MULTILINESTRING ((-122.55730 45.49583, -122.55...",9 to 20
1,0,3478,389,2171,918,512.015194,1,9,BUS,4562,SE Powell & 102nd,12757,NE 10th & Hood (Gresham Central TC),"LINESTRING (-122.55799 45.49578, -122.55797 45...",9 to 20
2,0,3478,389,2171,918,512.015194,3,20,BUS,2253,Gresham Central Transit Center,10879,"NE Kane & 29th, MHCC","LINESTRING (-122.42647 45.50264, -122.42633 45...",9 to 20
3,1,3615,392,2330,893,503.706955,0,WALK,WALK,,Origin,4562,SE Powell & 102nd,"MULTILINESTRING ((-122.55730 45.49583, -122.55...",9 to 73 to 20
4,1,3615,392,2330,893,503.706955,1,9,BUS,4562,SE Powell & 102nd,4573,SE Powell & 122nd,"LINESTRING (-122.55799 45.49578, -122.55797 45...",9 to 73 to 20
5,1,3615,392,2330,893,503.706955,3,73,BUS,6655,SE 122nd & Powell,6652,SE 122nd & Oak,"LINESTRING (-122.53767 45.49740, -122.53767 45...",9 to 73 to 20
6,1,3615,392,2330,893,503.706955,5,20,BUS,5425,SE Stark & 122nd,5397,SE Stark & NE Hale,"LINESTRING (-122.53724 45.51902, -122.53699 45...",9 to 73 to 20


In [1]:
import datetime 
import requests
mapbox_api_key = "XX"

In [5]:
def call_mapbox(fromPlace, toPlace):
    ''' 
    fromPlace = "lat, lon"
    toPlace = "lat,lon"

    mapbox takes lon, lat
    '''
    date = datetime.datetime.now().strftime("%Y-%m-%d")
    base_url = "https://api.mapbox.com/directions/v5/mapbox/driving-traffic/"
    time="12:00"
    depart_time=f"{date}T{time}"
    from_lon = fromPlace.split(",")[-1]
    from_lat = fromPlace.split(",")[0]
    to_lon = toPlace.split(",")[-1]
    to_lat = toPlace.split(",")[0]
    base_url_w_origin_dest = f"{base_url}{from_lon},{from_lat};{to_lon},{to_lat}"
    r = requests.get(url=base_url_w_origin_dest, params={'depart_at':depart_time
                                    ,'access_token':mapbox_api_key})
    assert(r.status_code==200)
    return r.json()

In [6]:
origin_str = "45.495953, -122.557314"
destination_str = "45.517341, -122.401294"

In [7]:
call_mapbox(origin_str, destination_str)

{'routes': [{'weight_typical': 1218.102,
   'duration_typical': 1026.988,
   'weight_name': 'auto',
   'weight': 1218.102,
   'duration': 1026.988,
   'distance': 20983.676,
   'legs': [{'via_waypoints': [],
     'admins': [{'iso_3166_1_alpha3': 'USA', 'iso_3166_1': 'US'}],
     'weight_typical': 1218.102,
     'duration_typical': 1026.988,
     'weight': 1218.102,
     'duration': 1026.988,
     'steps': [],
     'distance': 20983.676,
     'summary': 'I 205 North, I 84 East'}],
   'geometry': '_|ttGb~_kVNfK_Hj`@oKXevDrAcxAmUmScL{EyIo@i}@`_@c|FOalCnR_uEsA_i@yLwdAbJgs@|Boc@fm@H`CoEn@}Q`EqFtp@XvR}C@weAvIB'}],
 'waypoints': [{'distance': 12.781,
   'name': 'Southeast Powell Boulevard',
   'location': [-122.557296, 45.495839]},
  {'distance': 9.161,
   'name': 'Northeast Hale Avenue',
   'location': [-122.401179, 45.517326]}],
 'code': 'Ok',
 'uuid': 'HaMjNNT5ODs57Kr5meINw-0frXJ0xifzVzYVJbfJb5ZVl3v6DvsRug=='}